# 30 E Banda Sea

Authors: Cameron Seebeck, Kidus Teshome, Cian Wilson

## Steady state implementation

### Preamble

Set some path information.

In [ ]:
import sys, os
basedir = ''
if "__file__" in globals(): basedir = os.path.dirname(__file__)
sys.path.append(os.path.join(basedir, os.path.pardir))
sys.path.append(os.path.join(basedir, os.path.pardir, os.path.pardir, 'python'))

Loading everything we need from `sz_problem` and also set our default plotting and output preferences.

In [ ]:
import utils
from sz_base import allsz_params
from sz_slab import create_slab, plot_slab
from sz_geometry import create_sz_geometry
from sz_problem import SubductionProblem
import numpy as np
import dolfinx as df
import pyvista as pv
if __name__ == "__main__" and "__file__" in globals():
    pv.OFF_SCREEN = True
import pathlib
output_folder = pathlib.Path(os.path.join(basedir, "output"))
output_folder.mkdir(exist_ok=True, parents=True)

### Parameters

We first select the name and resolution scale, `resscale` of the model.

```{admonition} Resolution
By default the resolution is low to allow for a quick runtime and smaller website size.  If sufficient computational resources are available set a lower `resscale` to get higher resolutions and results with sufficient accuracy.
```


In [ ]:
name = "30_E_Banda_Sea"
resscale = 3.0
cfl      = 3.0

Then load the remaining parameters from the global suite.

In [ ]:
szdict = allsz_params[name]
print("{}:".format(name))
print("{:<20} {:<10}".format('Key','Value'))
print("-"*85)
for k, v in allsz_params[name].items():
    if v is not None and k not in ['z0', 'z15']: print("{:<20} {}".format(k, v))

### Setup

Setup a slab.

In [ ]:
slab = create_slab(szdict['xs'], szdict['ys'], resscale, szdict['lc_depth'])

Create the subduction zome geometry around the slab.

In [ ]:
geom = create_sz_geometry(slab, resscale, szdict['sztype'], szdict['io_depth'], szdict['extra_width'], 
                             szdict['coast_distance'], szdict['lc_depth'], szdict['uc_depth'])

### Solve steady state

Solve using a dislocation creep rheology and assuming a steady state.

In [ ]:
szss = SubductionProblem(geom, **szdict)
szss.solve_steadystate_dislocationcreep()

### Plot

Plot the solution.

In [ ]:
plotter = pv.Plotter()
utils.plot_scalar(szss.T_i, plotter=plotter, scale=szss.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot_vector_glyphs(szss.vw_i, plotter=plotter, gather=True, factor=0.1, color='k', scale=utils.mps_to_mmpyr(szss.v0))
utils.plot_vector_glyphs(szss.vs_i, plotter=plotter, gather=True, factor=0.1, color='k', scale=utils.mps_to_mmpyr(szss.v0))
utils.plot_geometry(geom, plotter=plotter, color='green', width=2)
utils.plot_couplingdepth(slab, plotter=plotter, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot_show(plotter)
utils.plot_save(plotter, output_folder / "{}_ss_solution_resscale_{:.2f}.png".format(name, resscale))

### Solve time-dependent

Solve using a dislocation creep rheology.

In [ ]:
sztd = SubductionProblem(geom, **szdict)

fps = 5
plotter = pv.Plotter(notebook=False, off_screen=True)
utils.plot_scalar(sztd.T_i, plotter=plotter, scale=sztd.T0, gather=True, cmap='coolwarm', clim=[0.0, sztd.Tm*sztd.T0], scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot_geometry(geom, plotter=plotter, color='green', width=2)
utils.plot_couplingdepth(slab, plotter=plotter, render_points_as_spheres=True, point_size=10.0, color='green')
plotter.open_gif( str(output_folder / "{}_td_solution_resscale_{:.2f}_cfl_{:.2f}.gif".format(name, resscale, cfl,)), fps=fps)

# Select the timestep based on the approximate target Courant number
dt = cfl*resscale/szdict['Vs']
# Reduce the timestep to get an integer number of timesteps
dt = szdict['As']/np.ceil(szdict['As']/dt)
sztd.solve_timedependent_dislocationcreep(szdict['As'], dt, theta=0.5, rtol=1.e-1, verbosity=1, plotter=plotter)

plotter.close()

### Plot

Plot the solution.

In [ ]:
plotter = utils.plot_scalar(sztd.T_i, scale=sztd.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature (deg C)', 'bold':True})
utils.plot_vector_glyphs(sztd.vw_i, plotter=plotter, gather=True, factor=0.1, color='k', scale=utils.mps_to_mmpyr(sztd.v0))
utils.plot_vector_glyphs(sztd.vs_i, plotter=plotter, gather=True, factor=0.1, color='k', scale=utils.mps_to_mmpyr(sztd.v0))
utils.plot_geometry(geom, plotter=plotter, color='green', width=2)
utils.plot_couplingdepth(slab, plotter=plotter, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot_show(plotter)
utils.plot_save(plotter, output_folder / "{}_td_solution_resscale_{:.2f}_cfl_{:.2f}.png".format(name, resscale, cfl,))

## Comparison

In [ ]:
T_diff = df.fem.Function(sztd.V_T)
T_diff.x.array[:] = szss.T_i.x.array - sztd.T_i.x.array
T_diff.name = 'T'

In [ ]:
plotter = utils.plot_scalar(T_diff, scale=szss.T0, gather=True, cmap='coolwarm', scalar_bar_args={'title': 'Temperature difference (deg C)', 'bold':True})
utils.plot_geometry(geom, plotter=plotter, color='green', width=2)
utils.plot_couplingdepth(slab, plotter=plotter, render_points_as_spheres=True, point_size=10.0, color='green')
utils.plot_show(plotter)
utils.plot_save(plotter, output_folder / "{}_ss-td_solution_resscale_{:.2f}_cfl_{:.2f}.png".format(name, resscale, cfl,))